# BERT

In [62]:
import pandas as pd

import transformers
from transformers import BertTokenizer

In [63]:
# in alternativa usare %%writefile per creare un file py e importare questo
import sys
sys.path.append('..')

import import_ipynb
from data_preparation import Preprocessing

In [64]:
df = pd.read_csv('../../data/updated_tweets.csv')

## Preprocessing

### Normalize

In [65]:
def normalize_tweet_BERT(tweet):
    tweet = Preprocessing.remove_links_mentions(tweet)
    tweet = tweet.lower()
    tweet = Preprocessing.remove_hashtag(tweet)
    tweet = Preprocessing.remove_special_characters(tweet)
 
    tweet = Preprocessing.remove_spaces(tweet)
    tweet = Preprocessing.remove_textual_emojis(tweet)
    tweet = Preprocessing.remove_not_ASCII(tweet)

    return tweet

In [66]:
df.tweet_text = [normalize_tweet_BERT(tweet) for tweet in df.tweet_text]

In [67]:
df = Preprocessing.clean_normalized_df(df)

### Dataset split

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Split del DataFrame in X (features) e y (etichette)
X = df['tweet_text']
y = df['cyberbullying_type']

# Split stratificato del dataset in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Esempio di vettorizzazione delle features dei tweet utilizzando TF-IDF
'''vectorizer = TfidfVectorizer(max_features=10000)  # Puoi regolare il numero massimo di features
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Addestramento del modello su X_train_vectorized e y_train
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

# Effettua le previsioni sul set di test
y_pred = model.predict(X_test_vectorized)

# Calcola le metriche di valutazione
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Stampa le metriche di valutazione
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1) 
X_train.head()'''

'vectorizer = TfidfVectorizer(max_features=10000)  # Puoi regolare il numero massimo di features\nX_train_vectorized = vectorizer.fit_transform(X_train)\nX_test_vectorized = vectorizer.transform(X_test)\n\n# Addestramento del modello su X_train_vectorized e y_train\nmodel = LogisticRegression()\nmodel.fit(X_train_vectorized, y_train)\n\n# Effettua le previsioni sul set di test\ny_pred = model.predict(X_test_vectorized)\n\n# Calcola le metriche di valutazione\naccuracy = accuracy_score(y_test, y_pred)\nprecision = precision_score(y_test, y_pred, average=\'weighted\')\nrecall = recall_score(y_test, y_pred, average=\'weighted\')\nf1 = f1_score(y_test, y_pred, average=\'weighted\')\n\n# Stampa le metriche di valutazione\nprint("Accuracy:", accuracy)\nprint("Precision:", precision)\nprint("Recall:", recall)\nprint("F1 Score:", f1) \nX_train.head()'

### Tokenize

In [69]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)           #cambiare nome bert-base-uncased?

In [70]:
'''encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in df.tweet_text] 
max_len = max([len(t) for t in encoded_tweets])
max_len'''
#la differenza è che tokenizer.encode restituisce una sequenza di interi codificati che rappresentano i token,
#includendo anche l'aggiunta di token speciali e altre operazioni di preprocessing.

'encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in df.tweet_text] \nmax_len = max([len(t) for t in encoded_tweets])\nmax_len'

In [71]:
tokens = [tokenizer.tokenize(tweet) for tweet in df.tweet_text]

### Padding

Cerchiamo la frase con più token

In [72]:
max_len = max([len(t) for t in tokens])
max_len

428

In [73]:
def lunghezza_media(tokens):
    return sum(len(array) for array in tokens) / len(tokens)

avg_len_tweet = lunghezza_media(tokens)
avg_len_tweet

32.62096424287282

In [74]:
'''for t in tokens:
    if len(t) == 428:
        print(t)'''

'for t in tokens:\n    if len(t) == 428:\n        print(t)'

In [75]:
max_length = 64

In [76]:
for i in range(len(tokens)):
    t = tokens[i][:max_length - 2]  # -2 per fare spazio ai token speciali [CLS] e [SEP]
    # Aggiungi token speciali
    t.insert(0, 'CLS')
    t.append('SEP')
    tokens[i] = t

print(tokens[1])

['CLS', 'why', 'is', 'aus', '##sie', '##tv', 'so', 'white', '?', 'SEP']


In [77]:
# tokens = [['CLS'] + t[:max_length - 2] + ['SEP'] for t in tokens]

### Creazione della maschera di attenzione

In [78]:
import torch


# Tokenizzazione e padding
padded_tokens = [t + ['[PAD]'] * (max_length - len(t)) for t in tokens]

# Creazione della maschera di attenzione
attention_mask = [[1] * len(t) + [0] * (max_length - len(t)) for t in tokens]

# Converti in tensori PyTorch
input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(t) for t in padded_tokens])
attention_mask = torch.tensor(attention_mask)

# Stampa l'input preparato
print(padded_tokens[1])
print(padded_tokens[2])
print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)

['CLS', 'why', 'is', 'aus', '##sie', '##tv', 'so', 'white', '?', 'SEP', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['CLS', 'a', 'class', '##y', 'whore', '?', 'or', 'more', 'red', 'velvet', 'cup', '##cake', '##s', '?', 'SEP', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [79]:
'''import torch
from transformers import BertModel

# Carica il modello pre-addestrato BERT
model = BertModel.from_pretrained('bert-base-uncased')

# Passa l'input attraverso il modello BERT per ottenere le rappresentazioni nascoste
outputs = model(input_ids, attention_mask=attention_mask)

# Estrai le rappresentazioni nascoste dell'ultimo layer
hidden_states = outputs.last_hidden_state

# Stampa le dimensioni delle rappresentazioni nascoste
print("Dimensioni delle rappresentazioni nascoste:", hidden_states.size())'''


'import torch\nfrom transformers import BertModel\n\n# Carica il modello pre-addestrato BERT\nmodel = BertModel.from_pretrained(\'bert-base-uncased\')\n\n# Passa l\'input attraverso il modello BERT per ottenere le rappresentazioni nascoste\noutputs = model(input_ids, attention_mask=attention_mask)\n\n# Estrai le rappresentazioni nascoste dell\'ultimo layer\nhidden_states = outputs.last_hidden_state\n\n# Stampa le dimensioni delle rappresentazioni nascoste\nprint("Dimensioni delle rappresentazioni nascoste:", hidden_states.size())'